This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_best_worst_split(y_true, best_run, worst_run, savepath,
                          title='Best Worst Overlay', label='target_value'):
    """
    Method to create a parity plot (predicted vs. true values) of just the best scoring and worst scoring CV splits

    Args:

        y_true: (numpy array), array of true y data

        best_run: (dict), the best scoring split_result from mastml_driver

        worst_run: (dict), the worst scoring split_result from mastml_driver

        savepath: (str), path to save plots to

        title: (str), title of the best_worst_split plot

        label: (str), label used for axis labeling

    Returns:

        None

    """
    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    maxx = max(y_true) # TODO is round the right thing here?
    minn = min(y_true)
    maxx = round(float(maxx), rounder(maxx-minn))
    minn = round(float(minn), rounder(maxx-minn))
    ax.plot([minn, maxx], [minn, maxx], 'k--', lw=2, zorder=1)

    # set tick labels
    _set_tick_labels(ax, maxx, minn)

    # do the actual plotting
    ax.scatter(best_run['y_test_true'],  best_run['y_test_pred'],  c='red',
               alpha=0.7, label='best test',  edgecolor='darkred',  zorder=2, s=100)
    ax.scatter(worst_run['y_test_true'], worst_run['y_test_pred'], c='blue',
               alpha=0.7, label='worst test', edgecolor='darkblue', zorder=3, s=60)
    ax.legend(loc='lower right', fontsize=12)

    # set axis labels
    ax.set_xlabel('True '+label, fontsize=16)
    ax.set_ylabel('Predicted '+label, fontsize=16)

    #font_dict = {'size'   : 10, 'family' : 'sans-serif'}

    # Duplicate the stats dicts with an additional label
    best_stats = OrderedDict([('Best Run', None)])
    best_stats.update(best_run['test_metrics'])
    worst_stats = OrderedDict([('worst Run', None)])
    worst_stats.update(worst_run['test_metrics'])

    plot_stats(fig, best_stats, x_align=x_align, y_align=0.90)
    plot_stats(fig, worst_stats, x_align=x_align, y_align=0.60)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([177.28302038, 150.37102595, 202.0924573 , ...,  47.20790294,
        23.809066  ,   0.        ])
best_run = OrderedDict([('normalizer', 'StandardScaler'), ('selector', 'SequentialFeatureSelector'), ('model', 'KernelRidge'), ('splitter', 'RepeatedKFold'), ('split_num', 9), ('y_train_true', array([177.28302038, 150.37102595, 202.0924573 , ...,   0.        ,
        47.20790294,   0.        ])), ('y_train_pred', array([236.48363759, 169.98335839, 144.91973485, ...,  66.92658557,
        55.51053098,   7.33894856])), ('y_test_true', array([2.22927321e+02, 2.14185752e+02, 1.82129033e+02, 3.96883335e+02,
       3.88395179e+02, 2.44558934e+02, 7.98843255e+01, 1.21202059e+02,
       1.43144270e+02, 1.10503751e+02, 1.29676557e+02, 1.06566685e+02,
       2.14252916e+02, 3.04216098e+02, 1.78590946e+02, 1.11989369e+02,
       1.47491884e+02, 9.77537153e+01, 2.33371776e+02, 1.88725323e+02,
       1.44367703e+02, 3.21795003e+02, 1.96095876e+02, 1.25081235e+02,
       1.37316410e+02, 1.72040539e+02, 1.85728826e+02, 1.14367926e+02,
       1.37488659e+02, 1.27740217e+02, 1.56941241e+02, 1.58367410e+02,
       1.21204438e+02, 4.57531836e+01, 2.16105345e+01, 2.89476692e+01,
       9.78662706e+01, 2.93743989e+02, 8.60496631e+01, 9.70462162e+01,
       1.11442294e+02, 6.36339325e+02, 1.56229730e+02, 2.36664682e+02,
       9.25437335e+01, 2.01817565e+02, 1.34180693e+02, 1.02036436e+02,
       1.82740912e+02, 2.38334229e+02, 7.34978297e+01, 7.14456318e+01,
       6.43729766e+02, 7.66704600e+00, 9.04068336e+01, 6.92869614e+01,
       6.36890317e+01, 1.14525327e+02, 5.00993000e+01, 0.00000000e+00,
       1.41664751e+02, 7.38023139e+01, 6.31512971e+01, 1.01797990e+02,
       0.00000000e+00, 1.05881855e+01, 2.26412639e+01, 2.99180823e+01,
       9.68586984e+01, 1.12003551e+02, 1.27239967e+02, 2.12302521e+02,
       1.51797441e+02, 2.89712978e+02, 2.51771744e+02, 1.50472708e+02,
       4.31928159e+01, 3.31384554e+01, 4.52159618e+01, 6.21679659e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.92094295e+00,
       1.99014786e+01, 0.00000000e+00, 1.69769380e+02, 1.39370004e+02,
       7.53897668e+01, 1.35168013e+02, 0.00000000e+00, 4.06187327e+01,
       3.81461657e+01, 6.05219124e+01, 5.67517336e+01, 4.89943291e+01,
       0.00000000e+00, 3.13433527e+01, 1.81910775e+02, 1.50508186e+02,
       1.08979858e+02, 8.64067635e+01, 1.14247314e+02, 1.15200294e+02,
       1.02514386e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.19314875e+01, 4.93632300e+01, 1.62043592e+01,
       0.00000000e+00, 1.48100516e+01, 5.26296357e+01, 6.06802766e+01,
       7.37896699e+00, 0.00000000e+00, 2.57222520e+01, 2.21842737e+01,
       9.55205228e+01, 3.73488338e+01, 0.00000000e+00, 6.12137026e+01,
       3.51232216e+01, 0.00000000e+00, 1.17714484e+02, 5.35428595e+01,
       1.12573845e+02, 2.68325976e+01, 5.86508558e+01, 1.29323672e+02,
       1.75579596e+02, 0.00000000e+00, 2.02289181e-01, 2.24784003e+01,
       1.37126059e+02, 7.31812963e+01, 1.25292208e+02, 0.00000000e+00,
       9.36717096e+01, 0.00000000e+00, 8.69178900e+00, 2.63839575e+02,
       2.76340269e+02, 8.39171957e+01, 2.51360287e+02, 4.73693459e+01,
       2.81102443e+02, 1.40538550e+02, 2.29064683e+02, 3.67624432e+02,
       1.99684569e+02, 4.96917625e+01, 2.53031123e+01, 0.00000000e+00,
       3.29759565e+01, 1.38924377e+01, 1.05110827e+01, 0.00000000e+00,
       0.00000000e+00, 2.15174731e+01, 5.12388154e+01, 0.00000000e+00,
       0.00000000e+00, 4.81001678e+01, 5.40734041e+01, 1.32773863e+02,
       2.04099993e+02, 7.67577252e+01, 3.81230146e+01, 3.74738040e+01,
       1.57158326e+02, 5.07080997e+01, 0.00000000e+00, 0.00000000e+00,
       3.42510607e+01, 3.13514481e+01, 1.52429429e+02, 8.15909270e+01,
       9.43019059e+01, 1.41373726e+02, 4.00673869e+01, 2.50852147e+01,
       2.67496464e+01, 2.85835223e+01, 6.33172566e+01, 3.63567935e+01,
       1.76733958e+01, 2.57130076e+01, 2.75959946e+01, 5.17559553e+01,
       8.22953902e+01, 7.16777317e+01, 1.01745118e+01, 4.94252903e+01,
       1.05442368e+02, 1.77187703e+01, 0.00000000e+00, 5.43550612e+00,
       1.78137720e+02, 1.89735033e+01, 2.11232453e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.61974495e+01,
       0.00000000e+00, 0.00000000e+00, 3.56929084e+01, 7.37535702e+01,
       1.42333954e+02, 2.44348237e+02, 1.55663046e+02, 1.57242981e+02,
       2.60129737e+02, 2.17252824e+02, 1.79654132e+02, 1.83828681e+02,
       1.65118221e+02, 2.75656178e+02, 2.87661839e+02, 3.67138367e+02,
       8.15746890e+01, 1.30390352e+02, 1.03756755e+02, 1.00207954e+02,
       2.59886084e+02, 1.96409134e+02, 1.44708713e+02, 1.22677657e+02,
       1.44006558e+02, 1.33954726e+02, 1.06220359e+02, 1.22431670e+02,
       1.21428977e+02, 2.31128112e+02, 1.18248560e+02, 1.77752168e+02,
       3.19116631e+02, 1.69573331e+02, 2.22246511e+02, 3.23822637e+02,
       2.04046571e+02, 1.30558998e+02, 2.47684975e+02, 2.52161354e+02,
       2.96095913e+02, 9.69267962e+01, 1.54641811e+02, 1.98592674e+02,
       3.54737752e+02, 3.53349574e+02, 3.38955480e+02, 3.30939913e+02,
       1.79999332e+02, 1.44889042e+02, 3.50052571e+02, 3.49605405e+02,
       2.18137285e+02, 2.23307827e+02, 7.10448978e+01, 1.44154498e+02,
       1.34599602e+02, 1.50042028e+02, 2.23624761e+02, 1.65109730e+02,
       6.45557199e+01, 6.15763632e+01, 1.50801863e+02, 1.26818170e+02,
       4.52932141e+01, 1.17792277e+02, 2.33383666e+02, 2.83042429e+02,
       1.32211588e+02, 2.51703483e+02, 1.61637650e+02, 9.69622803e+01,
       5.74165434e+01, 1.10065846e+02, 1.09913097e+02, 1.31798616e+02,
       1.22185191e+02, 1.02217058e+02, 2.66189733e+01, 1.74303139e+01,
       3.51009606e+01, 9.31208466e+01, 7.34595746e+01, 1.27664645e+02,
       4.18854297e+01, 3.05328625e+01, 1.82785945e+01, 3.31838908e+01,
       7.58528445e+01, 1.73730028e+02, 2.05075217e+02, 1.62590223e+02,
       1.76433607e+02, 1.47336842e+02, 1.98013501e+02, 1.67324805e+02,
       1.70292754e+02, 1.87207178e+02, 2.36861914e+02, 1.65127413e+02,
       1.33312148e+02, 1.11384136e+02, 1.42933388e+02, 1.79710356e+02,
       1.84787732e+02, 1.37406686e+02, 0.00000000e+00, 1.24542111e+02,
       1.59403546e+02, 8.35686242e+01, 1.37479653e+02, 1.09488956e+02,
       9.60360384e+01, 8.66997511e+01, 1.17249168e+02, 9.79185800e+01,
       1.83151200e+02, 1.71490363e+02, 1.15594393e+02, 1.35955483e+02,
       1.45310469e+02, 1.40364781e+02, 1.63497124e+02, 9.12669747e+01,
       8.76081324e+01, 9.37481720e+01, 7.98648319e+01, 6.69631063e+01,
       3.63667394e+01, 2.00619416e+01, 3.86328698e+01, 3.97988019e+01,
       5.91429461e+01, 7.14796192e+01, 7.26752642e+01, 6.16998797e+01,
       9.88464407e+01, 3.41670362e+02, 1.43739984e+02, 8.70884029e+01,
       9.43179777e+01, 1.82417160e+02, 1.79369514e+02, 3.81360533e+01,
       1.10110223e+02, 0.00000000e+00, 6.62624423e+01, 0.00000000e+00,
       1.12174952e+02, 7.18885960e+01, 1.24759381e+02, 1.98105436e+02,
       2.15058665e+01, 4.75557533e+00, 1.13483054e+02, 1.36252990e+02,
       6.23247673e+01, 5.63388158e+01, 5.63192894e+01, 5.92192098e+01,
       6.24481656e+01, 1.21974896e+02, 0.00000000e+00, 4.87758689e+01,
       4.34643306e+01, 1.24011708e+01, 2.35812529e+01, 2.71196539e+01,
       3.40153297e+01, 4.92018258e+01, 2.07757183e+02, 7.56574945e+01,
       8.29015442e+01, 6.98177552e+01, 6.74148765e+01, 7.04541654e+01,
       6.55202594e+01, 1.21693503e+02, 9.11059876e+01, 3.91108879e+01,
       4.54995709e+01, 4.50377785e+01, 4.82400402e+01, 3.39806022e+01,
       4.02393064e+01, 1.66347669e+02, 1.23233184e+02, 1.10664774e+02,
       1.05624926e+02, 1.42323884e+02, 3.15042600e+01, 3.74221989e+01,
       2.92307342e+01, 2.07374692e+01, 5.90626223e+01, 1.60395108e+02,
       4.26966290e+01, 1.20061041e+02, 7.00432397e+01, 3.53717988e+01,
       5.24254612e+01, 7.76829500e+01, 1.34709235e+01, 6.57797960e+01,
       1.21777150e+02, 2.19120080e+01, 7.17549475e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 8.66887169e+00, 0.00000000e+00,
       2.38090660e+01])), ('y_test_pred', array([ 2.05435371e+02,  1.68588256e+02,  1.70104105e+02,  2.67162467e+02,
        2.81622968e+02,  2.18397900e+02,  1.03053514e+02,  1.72764640e+02,
        1.16533832e+02,  1.21401439e+02,  8.67592637e+01,  9.22638256e+01,
        1.52548624e+02,  2.86865135e+02,  1.96733047e+02,  1.76668157e+02,
        1.17139826e+02,  9.84491588e+01,  2.32899624e+02,  1.69552534e+02,
        2.31937888e+02,  2.91922440e+02,  1.65442839e+02,  1.51356205e+02,
        1.53438827e+02,  1.98011096e+02,  1.91946267e+02,  1.51024681e+02,
        1.12957409e+02,  1.24900849e+02,  1.48144627e+02,  1.59775323e+02,
        1.30251639e+02,  6.99071307e+01,  3.75690526e+01,  5.07788965e+01,
        1.30662100e+02,  1.66647380e+02,  1.04557859e+02,  1.05975481e+02,
        1.18663890e+02,  5.82273429e+02,  1.70230724e+02,  2.26147366e+02,
        1.17667838e+02,  1.29451449e+02,  1.44766870e+02,  1.55199004e+02,
        1.77885773e+02,  1.71607797e+02,  6.95786543e+01,  7.31104904e+01,
        2.97897345e+02,  2.08228537e+01,  2.20981991e+02,  1.39970243e+02,
        1.96315339e+02,  1.99738624e+02,  3.97542671e+01,  2.05031922e+01,
        1.30559196e+02,  1.13120740e+02,  1.18084596e+02,  1.47585264e+02,
        3.45333504e+01,  4.18534742e+01,  5.10659722e+01, -8.67932125e-01,
        5.14057913e+01,  1.04841881e+02,  1.53435247e+02,  1.39561232e+02,
        1.47310979e+02,  1.69987991e+02,  1.65995511e+02,  1.61674807e+02,
        4.49857224e+01,  7.23596873e+01,  8.51948945e+01,  6.61610506e+01,
       -2.77959961e+00,  3.33767499e+01,  3.80570918e+01,  3.25093791e+01,
        5.48197201e+01, -4.56583036e+00,  2.48382700e+01,  2.62542249e+01,
        5.03770956e+01,  7.00501138e+01,  2.50055415e+01,  4.03814067e+01,
        2.61164442e+01,  4.08173149e+01,  1.45815523e+02,  1.33187388e+02,
        8.38051062e+01,  1.34099814e+02,  3.41173361e+01,  8.54660146e+01,
        6.70618023e+01,  4.80264889e+01,  3.91127243e+01,  4.32445129e+01,
        1.94869246e+01,  5.71949740e+01,  1.10593381e+02,  1.12916908e+02,
        9.85794651e+01,  8.84498310e+01,  9.01889685e+01,  9.29150486e+01,
        8.09485380e+01,  1.83126661e+01,  2.13944237e+01,  2.18272434e+01,
        1.99970705e+01,  4.48132454e+01,  7.20240973e+01,  8.29043259e+01,
        2.11255651e+01,  2.86918319e+01,  3.80521538e+01,  5.22835820e+01,
        2.61240445e+01,  2.75083692e+01,  2.47803790e+01,  2.19818999e+01,
        3.26817933e+01,  3.19638455e+01,  2.34702455e+01,  4.79991234e+01,
        4.75214812e+01,  3.71946986e+00,  5.60603631e+01,  7.15939820e+01,
        1.18676333e+02,  3.45067667e+01,  1.69740302e+01,  1.84724278e+02,
        2.36105620e+02,  3.37261893e+01,  2.05758425e+01,  3.74327242e+01,
        6.58655606e+01,  1.19398373e-04,  2.43957773e+02,  7.70389422e+01,
        1.19876981e+02,  3.92610863e+01,  3.67416764e+01,  2.75970271e+02,
        2.60142524e+02,  9.83335297e+01,  4.88321960e+01,  7.89413664e+01,
        2.01251581e+02,  9.50178194e+01,  1.06480484e+02,  2.13246447e+02,
        7.52983586e+01,  7.05393971e+01,  6.57020880e+01,  1.83433584e+01,
        6.15732522e+01,  3.67717262e+01,  1.91062285e+01,  1.41757402e+01,
        5.82289812e+00,  3.21238758e+01,  5.67103453e+01,  1.45339008e+01,
        2.10475361e+01,  4.21933346e+01,  2.89203246e+01,  1.12369736e+02,
        1.55147974e+02,  8.43680318e+01,  6.03426389e+01,  9.16526453e+01,
        6.91794791e+01,  5.92415812e+01, -2.14204698e+00,  4.20468750e-01,
        4.41026169e+01,  4.01961951e+01,  1.60416226e+02,  1.00705752e+02,
        8.67013994e+01,  8.29982145e+01,  3.50595968e+01,  3.27005424e+01,
        3.28668758e+01,  3.31471495e+01,  5.78804575e+01,  7.30288788e+01,
        2.47844734e+01,  4.73830455e+01,  5.17257108e+01,  4.88515106e+01,
        6.32251082e+01,  7.44002106e+01,  3.55411206e+01,  8.20599141e+00,
        3.18893902e+01,  3.70306429e+01,  3.49102918e+01,  3.23123765e+01,
        4.84033449e+01,  2.52473191e+01,  2.13318728e+01,  4.50711016e+01,
        2.58604664e+01,  1.14761255e+01,  1.16477893e+01,  1.38095346e+01,
        1.45931274e+01,  1.33917529e+01,  1.48827513e+01,  2.89103617e+01,
        1.76316547e+02,  2.77344700e+02,  1.43752687e+02,  1.32177310e+02,
        2.81424768e+02,  3.05281365e+02,  1.55111431e+02,  1.31235726e+02,
        1.25091048e+02,  1.70996941e+02,  2.66399095e+02,  3.01039821e+02,
        1.09595489e+02,  1.59126816e+02,  9.38359540e+01,  7.87823319e+01,
        1.70679433e+02,  1.36371218e+02,  1.32194501e+02,  8.41612078e+01,
        1.02746145e+02,  1.29947007e+02,  7.14494242e+01,  1.18594772e+02,
        1.21413454e+02,  1.49181279e+02,  1.39989982e+02,  1.48658473e+02,
        1.91366862e+02,  1.37759017e+02,  2.69463428e+02,  3.06056586e+02,
        2.17316621e+02,  1.63846519e+02,  2.41736858e+02,  1.67277915e+02,
        2.80130305e+02,  9.81280074e+01,  1.29896129e+02,  1.34844648e+02,
        3.09352692e+02,  2.52359328e+02,  2.36754908e+02,  2.88421635e+02,
        1.71407668e+02,  1.55746351e+02,  2.37599025e+02,  3.04294658e+02,
        1.65304581e+02,  1.87479299e+02,  8.48509054e+01,  1.71425449e+02,
        1.61298793e+02,  1.48668862e+02,  2.69861045e+02,  1.31489046e+02,
        8.07699083e+01,  6.91655725e+01,  8.44519889e+01,  1.19503262e+02,
        7.61181694e+01,  1.33357123e+02,  1.39541001e+02,  3.02629223e+02,
        1.65629831e+02,  2.97457241e+02,  1.39846183e+02,  1.27251936e+02,
        1.12912187e+02,  1.42191898e+02,  1.31311134e+02,  1.28514085e+02,
        1.37430305e+02,  1.28087995e+02,  7.03539248e+01,  3.90208352e+01,
        4.20025524e+01,  4.27501360e+01,  1.09380118e+02,  1.36031773e+02,
        5.58256500e+01,  7.20628355e+01,  3.82621773e+01,  3.90879176e+01,
        1.18846515e+02,  1.75703457e+02,  1.69305608e+02,  1.76322255e+02,
        1.81979201e+02,  1.80579908e+02,  1.82188805e+02,  1.16310489e+02,
        2.46352016e+02,  1.85084120e+02,  1.75624073e+02,  1.80369608e+02,
        1.30365125e+02,  1.16374775e+02,  1.33894211e+02,  1.76485888e+02,
        1.81077100e+02,  1.17769310e+02,  7.65152962e+01,  2.28796049e+02,
        1.35936569e+02,  1.25414869e+02,  1.59982787e+02,  1.34293301e+02,
        1.06164474e+02,  1.06597764e+02,  1.23117200e+02,  1.14459506e+02,
        1.40673414e+02,  1.12430616e+02,  1.14207618e+02,  1.43940021e+02,
        2.05032497e+02,  1.13255056e+02,  1.44175713e+02,  1.87604457e+02,
        1.08912820e+02,  1.25983708e+02,  1.00892328e+02,  7.30853337e+01,
        3.53327361e+01,  4.05552686e+01,  6.09552119e+01,  8.34967280e+01,
        8.64955577e+01,  9.64827936e+01,  9.89719781e+01,  1.05613916e+02,
        1.08106687e+02,  1.56294910e+02,  1.14620722e+02,  7.15937194e+01,
        1.19329714e+02,  1.17416653e+02,  1.50473031e+02,  6.11421105e+01,
        1.53859560e+02,  7.17721085e+00,  1.10416759e+02,  6.75572014e+00,
        4.65371006e+01,  7.84295792e+01,  1.09898530e+02,  1.52492789e+02,
        3.86475298e+01,  2.24117790e+01,  7.60247654e+01,  9.68763971e+01,
        7.10471198e+01,  8.61870044e+01,  5.34937772e+01,  9.90038207e+01,
        5.19542043e+01,  5.84833110e+01,  7.02356832e+00,  5.16279930e+01,
        3.70047913e+01,  3.46591469e+01,  2.28314779e+01,  3.18683866e+01,
        4.63802911e+01,  6.02898244e+01,  1.46006859e+02,  4.64907791e+01,
        5.59698568e+01,  7.30860612e+01,  9.05484117e+01,  4.49044313e+01,
        6.18058972e+01,  1.26177422e+02,  5.84323945e+01,  6.62663378e+01,
        3.10332707e+01,  3.53081357e+01,  5.19477788e+01,  3.79556217e+01,
        3.06631697e+01,  1.97613326e+02,  1.44924544e+02,  8.12027525e+01,
        7.28398255e+01,  7.87339223e+01,  2.67193555e+01,  2.23643764e+01,
        1.40681753e+01,  5.71865439e+01,  3.96227792e+01,  8.91059438e+01,
        1.97091245e+01,  5.25086832e+01,  3.87337544e+01,  4.62103936e+01,
        5.38015821e+01,  4.95854063e+01,  5.08914973e+01,  2.86009216e+01,
        8.36035526e+01,  5.12178482e+01,  3.15954537e+01,  1.05105557e+01,
        3.03319099e+01,  2.11618346e+01,  4.91328211e+01,  2.41388011e-01,
        4.48409355e+01])), ('train_metrics', OrderedDict([('R2', 0.7596954611738436), ('root_mean_squared_error', 47.402609232310056), ('mean_absolute_error', 32.21340207853417), ('rmse_over_stdev', 0.49418665079119495)])), ('test_metrics', OrderedDict([('R2', 0.7603765692707201), ('root_mean_squared_error', 46.1415397335229), ('mean_absolute_error', 31.89018694071966), ('rmse_over_stdev', 0.48103961685965824)])), ('train_indices', array([   0,    1,    2, ..., 2141, 2142, 2144])), ('test_indices', array([   8,   12,   13,   23,   26,   31,   33,   36,   38,   45,   49,
         50,   55,   63,   67,   68,   76,   81,   86,   91,   96,   99,
        107,  114,  116,  126,  131,  135,  139,  141,  153,  157,  158,
        162,  165,  169,  181,  192,  203,  204,  205,  211,  212,  213,
        231,  237,  241,  245,  247,  248,  250,  251,  257,  261,  268,
        270,  273,  280,  288,  291,  296,  297,  298,  301,  325,  326,
        327,  332,  338,  344,  345,  349,  356,  359,  363,  366,  381,
        383,  384,  399,  405,  411,  415,  418,  428,  434,  436,  441,
        442,  443,  444,  446,  448,  456,  461,  463,  474,  475,  480,
        491,  496,  498,  501,  510,  516,  519,  528,  535,  538,  539,
        540,  542,  543,  558,  560,  561,  562,  564,  565,  582,  584,
        585,  589,  590,  599,  600,  610,  612,  614,  615,  627,  636,
        641,  644,  651,  662,  668,  673,  675,  684,  687,  692,  693,
        695,  700,  701,  702,  708,  710,  717,  719,  739,  740,  742,
        750,  756,  765,  771,  772,  776,  778,  783,  792,  805,  808,
        814,  821,  825,  830,  837,  846,  862,  865,  866,  869,  874,
        875,  880,  892,  896,  899,  910,  912,  913,  925,  926,  950,
        953,  966,  968,  973,  976,  977,  978,  979,  982,  984,  987,
        988,  990,  991,  997, 1000, 1021, 1023, 1025, 1026, 1027, 1032,
       1035, 1038, 1039, 1042, 1055, 1056, 1059, 1060, 1061, 1064, 1067,
       1073, 1076, 1079, 1080, 1086, 1087, 1095, 1099, 1100, 1106, 1109,
       1110, 1126, 1155, 1158, 1159, 1164, 1175, 1178, 1182, 1184, 1185,
       1189, 1195, 1199, 1202, 1203, 1205, 1207, 1209, 1224, 1226, 1233,
       1260, 1275, 1279, 1280, 1286, 1288, 1290, 1323, 1327, 1332, 1333,
       1334, 1337, 1342, 1348, 1354, 1355, 1361, 1369, 1373, 1374, 1389,
       1399, 1402, 1411, 1412, 1413, 1415, 1430, 1432, 1438, 1445, 1458,
       1469, 1490, 1492, 1498, 1501, 1502, 1520, 1525, 1535, 1540, 1542,
       1545, 1556, 1560, 1561, 1564, 1567, 1571, 1574, 1589, 1590, 1591,
       1592, 1596, 1610, 1617, 1618, 1622, 1623, 1626, 1628, 1629, 1631,
       1639, 1640, 1647, 1650, 1654, 1658, 1659, 1664, 1669, 1672, 1673,
       1680, 1683, 1693, 1696, 1705, 1712, 1716, 1717, 1725, 1729, 1730,
       1732, 1735, 1743, 1751, 1766, 1771, 1777, 1778, 1784, 1785, 1790,
       1791, 1806, 1811, 1813, 1815, 1817, 1818, 1823, 1827, 1831, 1838,
       1844, 1848, 1849, 1850, 1851, 1853, 1854, 1856, 1857, 1858, 1861,
       1864, 1865, 1873, 1876, 1878, 1886, 1889, 1905, 1911, 1924, 1925,
       1930, 1947, 1948, 1957, 1961, 1962, 1972, 1974, 1977, 1979, 1981,
       1982, 1989, 1992, 1995, 2001, 2011, 2022, 2030, 2031, 2043, 2050,
       2056, 2060, 2064, 2065, 2067, 2069, 2073, 2076, 2082, 2084, 2097,
       2099, 2103, 2106, 2108, 2111, 2112, 2115, 2125, 2127, 2136, 2143])), ('train_groups', None), ('test_groups', None), ('prediction_metrics', None)])
worst_run = OrderedDict([('normalizer', 'StandardScaler'), ('selector', 'SequentialFeatureSelector'), ('model', 'KernelRidge'), ('splitter', 'RepeatedKFold'), ('split_num', 0), ('y_train_true', array([177.28302038, 150.37102595, 202.0924573 , ...,   0.        ,
        23.809066  ,   0.        ])), ('y_train_pred', array([218.49190694, 158.37366148, 143.26116819, ...,  67.13015962,
        48.82057995,   4.40653003])), ('y_test_true', array([2.18800571e+02, 2.39214774e+02, 2.41537703e+02, 2.19470531e+02,
       2.01402113e+02, 3.88395179e+02, 2.44558934e+02, 7.37475037e+01,
       1.40791488e+02, 9.96717956e+01, 2.41991216e+02, 2.14252916e+02,
       2.00047469e+02, 1.45936466e+02, 1.13270942e+02, 8.53211606e+01,
       1.01679243e+02, 2.33371776e+02, 1.81776188e+02, 1.91511615e+02,
       1.58791749e+02, 1.79995444e+02, 1.25772961e+02, 3.50019597e+02,
       3.06666897e+02, 2.29722452e+02, 1.72040539e+02, 1.03964994e+02,
       8.84608475e+01, 1.87427677e+02, 1.65793652e+02, 1.74631224e+02,
       1.68464956e+02, 1.31350152e+02, 3.79332462e+01, 2.16105345e+01,
       9.24493164e+01, 2.89476692e+01, 1.71608093e+02, 9.51442246e+01,
       2.36177315e+02, 2.54265050e+02, 2.91384616e+02, 5.15805330e+02,
       1.78235739e+02, 8.07377667e+01, 8.42025056e+01, 8.95029615e+01,
       1.36316077e+02, 1.03801111e+02, 9.24561079e+01, 0.00000000e+00,
       4.93432857e+01, 1.58218632e+02, 5.66738367e+01, 2.73135364e+01,
       1.56215250e+01, 1.01797990e+02, 5.64799569e+01, 1.77441804e+02,
       5.18644462e+01, 1.12003551e+02, 1.39599684e+02, 1.19778837e+02,
       2.18946397e+02, 2.53560446e+02, 2.02689680e+02, 1.50472708e+02,
       1.17650530e+02, 7.08357719e+01, 0.00000000e+00, 3.31384554e+01,
       4.09146349e+01, 5.57153253e+01, 2.58389489e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 9.45062097e+01, 1.69769380e+02, 7.39741468e+01,
       1.26328149e+01, 1.54429064e+01, 1.65750556e+01, 0.00000000e+00,
       3.34361123e+01, 7.74558848e+00, 0.00000000e+00, 3.21439514e+01,
       0.00000000e+00, 4.42073704e-01, 1.78001455e+02, 0.00000000e+00,
       1.33559658e+02, 1.40198319e+02, 1.50508186e+02, 1.14247314e+02,
       9.57636772e+01, 1.02514386e+02, 0.00000000e+00, 3.04689791e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.43690177e+01,
       8.99135404e+01, 0.00000000e+00, 5.53782777e+01, 0.00000000e+00,
       6.29442201e+01, 2.88123722e+01, 2.41335092e+01, 1.22038739e+02,
       4.48563810e+01, 2.61695184e+01, 2.91224504e+01, 3.66852424e+01,
       2.82257425e+01, 7.30016409e+01, 1.04542615e+02, 3.62570480e+01,
       5.86508558e+01, 3.50982856e+00, 1.26886312e+02, 2.78984665e+02,
       2.83906453e+02, 1.38892855e+02, 8.77843631e+01, 1.22311356e+02,
       0.00000000e+00, 0.00000000e+00, 1.30597392e+02, 3.29412986e+01,
       2.63839575e+02, 4.45414483e+01, 5.20286768e+01, 2.29064683e+02,
       1.34506442e+02, 2.99692485e+02, 1.99684569e+02, 4.96917625e+01,
       0.00000000e+00, 4.78013760e+01, 2.23740347e+01, 3.61695636e+01,
       2.62303388e+01, 3.36549838e+01, 1.05110827e+01, 0.00000000e+00,
       1.21655324e+01, 0.00000000e+00, 7.81817223e+01, 4.48488874e+01,
       4.49236078e+01, 4.71419734e+01, 3.08078053e+01, 1.70719435e+01,
       0.00000000e+00, 2.34800709e+02, 7.83502295e+01, 6.06316356e+01,
       1.10052006e+02, 1.57158326e+02, 5.02984410e+01, 3.32733667e+01,
       0.00000000e+00, 3.95429854e+01, 4.16645268e+01, 2.82496436e+01,
       3.13514481e+01, 5.20549201e+01, 3.92079232e+01, 2.14364169e+02,
       1.24729865e+02, 1.28386129e+02, 9.46538849e+01, 1.76733958e+01,
       1.03527711e+01, 3.36737583e+01, 1.15904985e+01, 6.67613331e+01,
       4.23440436e+01, 1.03266329e+02, 8.58908577e+01, 5.43550612e+00,
       3.57280662e+01, 0.00000000e+00, 9.36490231e+00, 6.90074178e+01,
       0.00000000e+00, 0.00000000e+00, 1.33030325e+02, 1.42333954e+02,
       1.68517827e+02, 1.36431444e+02, 1.82075421e+02, 2.45692439e+02,
       2.17252824e+02, 1.89093084e+02, 2.34660651e+02, 1.57972803e+02,
       2.13314660e+02, 3.78453371e+02, 2.87661839e+02, 3.42040019e+02,
       3.36224697e+02, 1.06514784e+02, 1.34673117e+02, 1.13869162e+02,
       1.81295285e+02, 2.49724729e+02, 1.31761462e+02, 1.86026530e+02,
       1.26700822e+02, 1.63799175e+02, 1.69625113e+02, 1.52280199e+02,
       1.29342617e+02, 1.22431670e+02, 1.77752168e+02, 1.69573331e+02,
       2.92536084e+02, 2.67340580e+02, 2.62896301e+02, 2.22246511e+02,
       2.06364164e+02, 3.20529825e+02, 3.18399908e+02, 2.01321423e+02,
       2.32699819e+02, 1.29319114e+02, 1.80660958e+02, 1.30558998e+02,
       8.23163755e+01, 9.69915119e+01, 8.86267599e+01, 2.47684975e+02,
       1.61293351e+02, 2.08659157e+02, 2.47005095e+02, 9.69267962e+01,
       1.93969856e+02, 1.87885255e+02, 1.88513164e+02, 1.92010418e+02,
       2.13333228e+02, 3.54737752e+02, 3.29042834e+02, 1.44889042e+02,
       2.47474247e+02, 8.54936441e+01, 1.24842572e+02, 1.44154498e+02,
       6.84009027e+01, 5.59498437e+01, 1.70005981e+02, 1.38575932e+02,
       1.48873887e+02, 1.09381016e+02, 9.73721500e+01, 5.55045542e+01,
       1.29155595e+02, 4.52932141e+01, 5.68526124e+01, 1.43519281e+02,
       1.99578096e+02, 2.33383666e+02, 2.72871580e+02, 1.84420825e+02,
       1.32211588e+02, 3.04851246e+02, 2.28169699e+02, 1.73746536e+02,
       2.04554894e+02, 2.04542101e+02, 2.09325439e+02, 1.42416121e+02,
       8.21190201e+01, 9.69622803e+01, 5.74165434e+01, 5.19883620e+01,
       1.51089123e+02, 9.54271066e+01, 6.28478449e+01, 1.89038755e+01,
       2.66189733e+01, 1.74303139e+01, 3.31481761e+01, 1.27664645e+02,
       2.20249128e+01, 4.28892788e+01, 5.66837962e+01, 7.59835817e+01,
       4.29043523e+01, 7.75933817e+01, 1.73730028e+02, 1.62590223e+02,
       1.76433607e+02, 1.73385595e+02, 1.88271436e+02, 2.10616630e+02,
       2.00391998e+02, 1.22370956e+02, 9.06089707e+01, 1.13060644e+02,
       1.83271284e+02, 1.17249168e+02, 1.26798490e+02, 2.96630230e+02,
       1.85661694e+02, 1.64660391e+02, 3.97239965e+02, 1.12287386e+02,
       1.09297300e+02, 9.87844184e+01, 1.38582903e+02, 1.40364781e+02,
       1.36808521e+02, 3.35321293e+02, 8.86858956e+01, 7.98648319e+01,
       6.97752946e+01, 2.63543371e+02, 1.07783426e+02, 5.65113296e+01,
       3.33198729e+01, 1.83475519e+02, 2.86171985e+01, 4.69172371e+01,
       2.71330052e+01, 1.18947119e+02, 1.51299348e+01, 3.73225941e+01,
       5.97413320e+01, 1.09582964e+02, 3.97988019e+01, 5.04471564e+01,
       7.14796192e+01, 7.26752642e+01, 7.84670662e+01, 1.34244899e+02,
       4.06890701e+01, 6.16998797e+01, 9.38114347e+01, 3.41670362e+02,
       3.87496682e+01, 0.00000000e+00, 9.82564416e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 6.41409237e+01, 1.98105436e+02,
       1.13483054e+02, 6.23247673e+01, 5.63192894e+01, 5.41514922e+01,
       5.32856309e+01, 4.94793928e+01, 7.26725204e+01, 7.32347672e+01,
       1.01755733e+02, 2.12813897e+01, 3.38853640e+01, 9.31924847e+00,
       1.24011708e+01, 3.23684503e+01, 4.56348093e+01, 3.17633706e+01,
       2.99157066e+01, 3.77846853e+01, 1.72121127e+01, 2.01387280e+01,
       7.56574945e+01, 7.65032664e+01, 7.72377141e+01, 6.05410832e+01,
       1.66797820e+02, 7.05986967e+01, 1.21693503e+02, 4.54995709e+01,
       0.00000000e+00, 4.44093613e+01, 4.82400402e+01, 3.39806022e+01,
       3.91274706e+01, 7.33023683e+01, 8.25826337e+01, 7.09430276e+01,
       3.76093386e+01, 3.62234796e+01, 9.01656399e+01, 4.11379073e+01,
       2.72516127e+01, 3.15042600e+01, 3.07458097e+01, 1.20865840e+02,
       4.59371875e+01, 2.93690973e+01, 9.40057842e+01, 6.65012486e+01,
       4.54076116e+01, 5.19044272e+01, 3.87785474e+01, 5.24254612e+01,
       4.50623816e+01, 8.97135102e+01, 1.25245007e+02, 5.97432500e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.88957798e+01, 0.00000000e+00, 0.00000000e+00,
       4.72079029e+01])), ('y_test_pred', array([190.31006537, 225.96890127, 162.93271385, 154.06887399,
       151.96269007, 273.732219  , 212.10310946, 138.35511192,
       112.73210861,  86.78179531, 247.63465254, 147.16911865,
       149.94510528, 137.29526552, 136.75464259, 100.35045099,
        92.22025578, 232.79564154, 171.22970082, 291.43658628,
       151.62354036, 150.91421436, 143.66045609, 242.10404226,
       267.49859505, 316.26351345, 193.74227663, 106.74235832,
        85.4196551 , 148.80561093, 151.58512525, 147.08077793,
       196.02173355,  28.32704865,  88.78013868,  28.23790509,
        91.63903771,  45.54424914, 127.69659012,  80.32859548,
        85.76819945,  87.64310448, 193.10182142, 509.94537352,
        83.4228358 , 129.05105773, 125.28366659, 108.34441892,
       140.84422391, 135.4346509 ,  85.02041234, 104.29525227,
       195.92586257, 115.08481155, 143.27115687,  77.06134967,
       106.32978154, 143.2901173 ,  60.96468024,  78.36695873,
        65.31785766, 122.3529941 , 138.51200477, 139.67293224,
       164.15363414, 220.82244079, 168.2218547 , 153.4316764 ,
        96.02719439,  78.65591276,   6.14073499,  76.18337433,
        59.20102592,  82.07971081,  59.89210155,  -3.40358094,
         6.79934582,  37.40910176,  32.96139774,  24.01026436,
        24.35412117,  59.76396725,  35.05694041,  16.8589657 ,
        24.5526336 ,  25.52541254,  21.88445442,  42.52723145,
        46.89618779,  92.42156665, 145.90317532,  82.62484821,
        33.0177533 ,  39.72195581,  65.01431996,  36.27282904,
        42.6130736 ,  44.58336869,  -2.24703185,  50.6468284 ,
        28.84582844,  64.38292421, 170.91768313,  13.53629712,
       106.20159193, 103.80207786, 106.43929438,  96.11338849,
        86.38644646,  87.26376865,  20.01984235,  42.1324558 ,
        14.42686619,  14.80619743,  15.74638193,  49.89622835,
        93.99871929,  35.60360849,  36.05927583,  26.82041953,
        32.77781787,  47.62718344,  21.72839476,  74.7871835 ,
        17.99080873,   8.49810018,  10.92807246,  30.57667152,
        36.27988252,  63.5743238 ,  59.83720905,   5.31871838,
        15.95701666,  33.80121802, 158.49803795, 330.24567551,
       232.58462996,  92.18487941, 125.7717719 , 114.23394067,
        93.79713934,   0.        ,  80.25623702,  68.72490929,
       287.14266998, 129.18000057,  32.20499401, 106.40399237,
        66.47867619, 211.86918872,  76.38070454,  66.88482866,
         7.45439608,  53.18926053,  42.67223043,  42.69966506,
        44.48799451,  44.8751601 ,  23.25585951,  12.14336248,
        39.48613349,  22.2215469 ,  70.78841786,  46.26491232,
        46.83668005,  47.950501  ,  36.75876122,  27.91556311,
        -4.17008923, 160.22342956,  85.23249936, 101.09239593,
        66.02130115,  71.23336537,  69.65200099,  63.75862708,
        -2.80922219,  51.40845101,  55.05178677,  64.3873969 ,
        41.03599482,  35.41989897,  50.08917496, 266.98727562,
       109.93882274, 110.05876872,  90.8593222 ,  24.30027701,
        28.10377619,  25.65298759,  34.18221169,  44.08457614,
        41.76404352,  37.9385526 ,  43.27991951,  28.79301859,
         3.17660904,  15.85310227,  42.00601231,  12.35592084,
        14.26355583,  13.85946358, 213.39445314, 179.3680015 ,
       142.52109003,  19.38562096, 161.4044429 , 310.19263592,
       305.20136221, 194.15554552, 223.83838258, 118.35324943,
       153.76469997, 308.83305725, 265.85047628, 276.0134192 ,
       257.61523563, 142.71556054, 210.6510166 , 158.82918555,
       254.88067904, 164.55523692, 103.27351341, 199.15230613,
       186.05694296, 182.91728612, 202.8056389 ,  98.98692803,
       124.93006433, 120.49942312, 154.19585641, 140.32290639,
       145.44397733, 309.10122633, 253.97194901, 268.82302666,
       227.8107564 , 229.0029697 , 268.53210568, 176.63913751,
       215.14550614,  87.74866009, 205.49058395, 163.97137708,
        82.84356976,  81.75902833, 131.55294844, 241.2401896 ,
       167.57632012, 224.17866693, 261.88052739,  93.25645177,
        96.84663603,  89.48185845,  75.42188316, 138.34274123,
       161.36371572, 310.23762722, 308.17980676, 157.81396547,
       204.93731891,  90.82862018, 184.89176956, 171.72721625,
        83.75690158,  84.04774674,  85.95718291, 124.26432745,
       205.71677396, 138.92608065, 145.51574534,  90.62501848,
       111.97210866,  69.50651967,  66.12152854,  73.27568899,
       172.80695798, 139.01097998, 309.40450467, 222.6920204 ,
       167.53808879, 238.50310292, 278.0062831 , 175.69780885,
       201.77510189, 222.4294716 , 158.25198268, 139.20274151,
       123.67644812, 125.67845941, 112.87045956, 114.14584339,
       147.94291067, 121.29296055,  71.18241149,  77.67422466,
        71.89505883,  22.92892517,  25.00129473, 138.91410275,
        34.37367086,  42.61376622,  43.0477209 ,  71.23594543,
        23.81795586,  43.0323279 , 181.71926405, 167.73130807,
       174.89516072, 158.90990958, 171.46538427, 174.3206443 ,
       188.55000618, 123.69171345, 126.50807041, 115.19907111,
       199.32768553, 124.69875822, 146.02268591, 310.2562208 ,
       124.2025793 , 131.18828372, 312.26975728, 104.54497468,
       102.0237248 , 101.78892258, 109.97043732, 113.44366635,
       126.50663769, 312.28235559, 111.18604225,  98.24394466,
        98.77163573, 311.47355253, 145.16608271,  63.22051544,
        35.49573555,  99.48067435,  36.85892809,  49.40520194,
        39.52181894,  83.88832642,  45.79563109,  41.42577844,
        58.2918433 , 101.97854087,  86.32267497,  96.25146831,
        97.80978565,  93.59403384, 101.71878067, 105.73925497,
        84.75370262, 104.00332855,  79.31736742, 180.42904751,
        55.49836203,   8.62472049,  55.28576086,  22.97304518,
         6.27938337,   6.51259016,  58.03115499, 147.52922809,
        78.90453478,  75.12113308,  40.70268938,  49.69842518,
        43.39115595,  43.89723624,  76.4673982 ,  57.79529879,
        47.77452003,  -0.82735138,  11.5292014 ,  11.65173694,
        37.97225288,  26.08690201,  56.28650392,  28.30600666,
        26.32107973,  36.15658738,  26.48498955,  29.1643135 ,
        49.19825656,  51.83144933,  44.92226988,  45.53975071,
        63.75349284,  52.38226235, 128.33543098,  30.70480029,
        -2.41969044,  39.35629836,  51.63268926,  38.96314006,
        31.0337008 ,  89.44822703,  83.96662346,  83.83283867,
        64.78112675,  57.47201787,  82.84501558,  27.71701216,
        25.3739043 ,  21.00865899,  23.53397787,  89.78930484,
        58.00693835,   9.21696434,  57.08107197,  33.7974346 ,
        33.90852966,  45.44641518,  30.73395536,  44.76704779,
        27.15022285,  62.32851783,  73.21003978,  17.55490225,
         7.93268951,  15.67222943,  16.50671609,   8.90386518,
        29.71196128,  45.16412504,  33.54985704,  -3.02953029,
        61.28474602])), ('train_metrics', OrderedDict([('R2', 0.7625207437033727), ('root_mean_squared_error', 47.59156345050221), ('mean_absolute_error', 32.27457536932615), ('rmse_over_stdev', 0.4918620510346152)])), ('test_metrics', OrderedDict([('R2', 0.7833798140469362), ('root_mean_squared_error', 41.87814445528851), ('mean_absolute_error', 30.229951584330216), ('rmse_over_stdev', 0.4328134764205723)])), ('train_indices', array([   0,    1,    2, ..., 2141, 2143, 2144])), ('test_indices', array([   9,   10,   14,   17,   21,   26,   31,   32,   40,   46,   54,
         55,   59,   62,   69,   78,   82,   86,   90,  100,  108,  111,
        117,  119,  120,  124,  126,  136,  146,  151,  152,  154,  156,
        160,  163,  165,  168,  169,  184,  188,  190,  200,  201,  215,
        227,  229,  230,  232,  235,  240,  264,  267,  274,  276,  284,
        290,  292,  301,  313,  337,  339,  344,  346,  355,  357,  358,
        362,  366,  370,  374,  376,  383,  387,  396,  397,  405,  408,
        416,  417,  422,  423,  426,  437,  438,  444,  450,  453,  456,
        457,  460,  461,  462,  470,  471,  477,  480,  488,  489,  500,
        507,  517,  521,  525,  529,  533,  534,  535,  540,  541,  543,
        544,  552,  554,  560,  572,  576,  577,  581,  593,  600,  606,
        609,  611,  616,  624,  631,  633,  638,  643,  650,  652,  653,
        675,  681,  686,  688,  690,  694,  709,  711,  715,  718,  720,
        721,  739,  743,  749,  772,  773,  777,  778,  783,  787,  793,
        797,  801,  806,  812,  821,  822,  826,  836,  843,  852,  853,
        854,  860,  861,  864,  873,  879,  889,  898,  899,  906,  908,
        912,  916,  918,  920,  926,  929,  935,  937,  947,  948,  954,
        984,  985,  993, 1001, 1004, 1006, 1009, 1022, 1027, 1030, 1041,
       1048, 1053, 1055, 1056, 1070, 1073, 1077, 1081, 1082, 1084, 1087,
       1092, 1097, 1102, 1105, 1107, 1109, 1111, 1113, 1118, 1121, 1127,
       1132, 1135, 1139, 1142, 1150, 1167, 1172, 1179, 1190, 1195, 1205,
       1209, 1210, 1214, 1218, 1224, 1227, 1235, 1241, 1242, 1243, 1254,
       1257, 1260, 1261, 1264, 1274, 1275, 1277, 1278, 1283, 1286, 1294,
       1297, 1302, 1303, 1308, 1323, 1326, 1337, 1349, 1362, 1368, 1369,
       1370, 1375, 1377, 1378, 1379, 1383, 1386, 1398, 1414, 1415, 1416,
       1417, 1428, 1432, 1433, 1441, 1445, 1452, 1453, 1455, 1460, 1461,
       1470, 1471, 1484, 1490, 1492, 1493, 1506, 1511, 1532, 1534, 1535,
       1540, 1543, 1560, 1568, 1569, 1572, 1577, 1585, 1587, 1589, 1591,
       1592, 1603, 1609, 1611, 1637, 1644, 1660, 1667, 1677, 1680, 1681,
       1686, 1692, 1694, 1698, 1702, 1709, 1710, 1715, 1717, 1720, 1724,
       1731, 1735, 1736, 1737, 1738, 1745, 1747, 1749, 1750, 1754, 1757,
       1759, 1764, 1770, 1772, 1775, 1777, 1783, 1784, 1785, 1787, 1788,
       1789, 1790, 1799, 1806, 1822, 1833, 1835, 1842, 1843, 1844, 1846,
       1851, 1856, 1858, 1864, 1866, 1868, 1869, 1870, 1872, 1887, 1895,
       1898, 1899, 1905, 1912, 1918, 1926, 1928, 1929, 1934, 1943, 1948,
       1952, 1956, 1966, 1971, 1973, 1977, 1982, 1985, 1991, 1992, 1995,
       2000, 2019, 2020, 2024, 2029, 2032, 2036, 2045, 2047, 2050, 2051,
       2055, 2057, 2059, 2066, 2075, 2079, 2080, 2081, 2084, 2085, 2091,
       2092, 2102, 2112, 2113, 2123, 2124, 2130, 2133, 2134, 2136, 2142])), ('train_groups', None), ('test_groups', None), ('prediction_metrics', None)])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/learning_curves/learning/StandardScaler/SequentialFeatureSelector/KernelRidge/RepeatedKFold/best_worst_split.png'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_best_worst_split(y_true, best_run, worst_run, savepath, label)
display(Image(filename='best_worst_split.png'))
